# Investigating the output of neural net embedding subnets - with 2D subnet output

## Aims

* To investigate the output of the hospital and clinical subnets of the embedding neural network.

* To examine the link between hospital subnet output and use of thrombolysis in hospitals - both the actual thrombolysis use, and the predicted thrombolysis use of a 10k set of patients passed through all hopsital moodels.

* To examine the link between the patient clinical feature subnet output and the use of thrombolysis, and the link between patient features and the clinical feature subnet output

## Neural Network structure

The model contains three subnets that take portions of the data. The output of these subnets is an n-dimensional vector. In this case the output is a 2D vector, that is each subnet is reduced to a single value output. The subnets created are for:

1. *Patient clinical data*: Age, gender, ethnicity, disability before stroke, stroke scale data. Pass through one hidden layer (with 2x neurons as input features) and then to single neuron with sigmoid activation.

2. *Pathway process data*: Times of arrival and scan, time of day, day of week. Pass through one hidden layer (with 2x neurons as input features) and then to single neuron with sigmoid activation.

3. *Hospital ID* (one-hot encoded): Connect input directly to single neuron with sigmoid activation.

The outputs of the three subnet outputs are then passed to a single neurone with sigmoid activation for final output.

![](./images/subnets_1.png)

## Fitting of model

The model has been pretrained (see the notebook *Modular TensorFlow model with 1D embedding - Train and save model for 10k patient subset*)

## Load libraries

In [1]:
# Turn warnings off to keep notebook tidy
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import pandas as pd

# sklearn for pre-processing
from sklearn.preprocessing import MinMaxScaler

# TensorFlow api model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy

## Define function to scale data

Scale input data 0-1 (MinMax scaling).

In [2]:
def scale_data(X_train, X_test):
    """Scale data 0-1 based on min and max in training set"""
    
    # Initialise a new scaling object for normalising input data
    sc = MinMaxScaler()

    # Set up the scaler just on the training set
    sc.fit(X_train)

    # Apply the scaler to the training and test sets
    train_sc = sc.transform(X_train)
    test_sc = sc.transform(X_test)
    
    return train_sc, test_sc

## Get model outputs for test data

Get prediction probabilities for the test 10k training set. Training data is used only to scale test set X values.

This prediction run is used to check model, and get accuracy.

In [3]:
# Load data
train = pd.read_csv(f'../data/10k_training_test/cohort_10000_train.csv')
test = pd.read_csv(f'../data/10k_training_test/cohort_10000_test.csv')

all_test = test.copy()

# Get data subgroups
subgroups = pd.read_csv('../data/subnet.csv', index_col='Item')
# Get list of clinical items
clinical_subgroup = subgroups.loc[subgroups['Subnet']=='clinical']
clinical_subgroup = list(clinical_subgroup.index)
# Get list of pathway items
pathway_subgroup = subgroups.loc[subgroups['Subnet']=='pathway']
pathway_subgroup = list(pathway_subgroup.index)
# Get list of hospital items
hospital_subgroup = subgroups.loc[subgroups['Subnet']=='hospital']
hospital_subgroup = list(hospital_subgroup.index)

# OneHot encode stroke team
coded = pd.get_dummies(train['StrokeTeam'])
train = pd.concat([train, coded], axis=1)
train.drop('StrokeTeam', inplace=True, axis=1)
coded = pd.get_dummies(test['StrokeTeam'])
test = pd.concat([test, coded], axis=1)
test.drop('StrokeTeam', inplace=True, axis=1)

# Split into X, y
X_train_df = train.drop('S2Thrombolysis',axis=1) 
y_train_df = train['S2Thrombolysis']
X_test_df = test.drop('S2Thrombolysis',axis=1) 
y_test_df = test['S2Thrombolysis'] 

# Split train and test data by subgroups
X_train_patients = X_train_df[clinical_subgroup]
X_test_patients = X_test_df[clinical_subgroup]
X_train_pathway = X_train_df[pathway_subgroup]
X_test_pathway = X_test_df[pathway_subgroup]
X_train_hospitals = X_train_df[hospital_subgroup]
X_test_hospitals = X_test_df[hospital_subgroup]

# Convert to NumPy
X_train = X_train_df.values
X_test = X_test_df.values
y_train = y_train_df.values
y_test = y_test_df.values

# Scale data
X_train_patients_sc, X_test_patients_sc = \
    scale_data(X_train_patients, X_test_patients)

X_train_pathway_sc, X_test_pathway_sc = \
    scale_data(X_train_pathway, X_test_pathway)

X_train_hospitals_sc, X_test_hospitals_sc = \
    scale_data(X_train_hospitals, X_test_hospitals)

# Load model
path = './saved_models/2d_for_10k/'
filename = f'{path}model.h5'
model = keras.models.load_model(filename)

# Test model
probability = model.predict(
    [X_test_patients_sc, X_test_pathway_sc, X_test_hospitals_sc])
y_pred_test = probability >= 0.5
y_pred_test = y_pred_test.flatten()
accuracy_test = np.mean(y_pred_test == y_test)
print(f'Accuracy test {accuracy_test:0.3f}')

all_test['model_prob'] = probability.flatten()
all_test['prediction'] = y_pred_test

Accuracy test 0.843


## Get predictions for thrombolysis use of 10k set of patients at each hospital

Here we ask the counter-factual question - "what treatment would a patient be expected to receive at each of the 132 hospitals?".

Hospial is one-hot encoded as input to the hospital subnet. To make a prediction of treatment at different hospitals we change the one-hot encoding of the hospital when making prediction.

For each hospital we pass through the 10k test set, and record the proportion of the patients receiving thrombolysis at that hospital.

In [ ]:
# Get number of hospitals
num_hospitals = len(X_test_hospitals_sc[0])
# Create test array for changing hospital ID
X_hospitals_alter = X_test_hospitals_sc.copy()
# List for all patient classifications at each hospital
patient_results = []

# Loop through setting hospital
hospital_results = []
for hosp in range(num_hospitals):
    # Set all hospitals to zero
    X_hospitals_alter[:,:] = 0
    # Set test hospital to 1
    X_hospitals_alter[:,hosp] = 1
    # Get probability of thrombolysis
    probability = model.predict(
        [X_test_patients_sc, X_test_pathway_sc, X_hospitals_alter])
    # Classify
    classified = probability >= 0.5
    # Get average thrombolysis (we are not stroring all individual results)
    thrombolysis = classified.mean()
    hospital_results.append(thrombolysis)
    patient_results.append(classified)

# Put results in DataFrame 
predicted_thrombolysis = pd.DataFrame()
predicted_thrombolysis['hospital'] = hospital_subgroup
predicted_thrombolysis['10k_thrombolysis'] = hospital_results

# Show DataFrame
predicted_thrombolysis

In [ ]:
patient_thromb_results = np.sum(patient_results, axis=0).flatten()
all_test['num_hosp_thrombolysing'] = patient_thromb_results
all_test

## Get subnet outputs

### Get hospital subnet output

In [ ]:
results = []
for hosp in range(num_hospitals):
    # Set all hospitals to zero
    X_hospitals_alter[:,:] = 0
    # Set test hospital to 1
    X_hospitals_alter[:,hosp] = 1
    # Get hospital subnet output
    layer_name = 'hospital_encode'
    hospital_encode_model = keras.Model(
        inputs=model.input,outputs=model.get_layer(layer_name).output)
    hospital_encode_output = hospital_encode_model([
        X_test_patients_sc, X_test_pathway_sc, X_hospitals_alter])
    results.append(np.mean(hospital_encode_output.numpy(),axis=0))

results = np.array(results)
hospital_encoding = pd.DataFrame(index=hospital_subgroup)
hospital_encoding['hosp_encode_x'] = results[:, 0]
hospital_encoding['hosp_encode_y'] = results[:, 1]
all_test = all_test.merge(
    hospital_encoding, left_on='StrokeTeam', right_index=True, how='left')

### Get patient subnet output

In [ ]:
# Get patient subnet output
layer_name = 'patient_encode'
patient_encode_model = keras.Model(
    inputs=model.input,outputs=model.get_layer(layer_name).output)
patient_encode_output = patient_encode_model([
    X_test_patients_sc, X_test_pathway_sc, X_test_hospitals_sc])
patient_encode_output = patient_encode_output.numpy()

In [ ]:
patient_encode_output

In [ ]:
all_test['patient_encode_x'] = patient_encode_output[:, 0]
all_test['patient_encode_y'] = patient_encode_output[:, 1]

### Get pathway subnet output

In [ ]:
# Get patient subnet output
layer_name = 'pathway_encode'
pathway_encode_model = keras.Model(
    inputs=model.input,outputs=model.get_layer(layer_name).output)
pathway_encode_output = pathway_encode_model([
    X_test_patients_sc, X_test_pathway_sc, X_test_hospitals_sc])
pathway_encode_output = pathway_encode_output.numpy()

pathway_encode_output

In [ ]:
all_test['pathway_encode_x'] = pathway_encode_output[:, 0]
all_test['pathway_encode_y'] = pathway_encode_output[:, 1]
all_test.to_csv('./output/2d_outputs.csv')
all_test

## Comparing hospital subnet activation with thrombolysis use at each hospital

The hospital subnet outputs a single value (in the range 0-1) for each hospital.

Here we compare the hospital subnet output for each hospital with:

1. The actual use of thrombolysis for 10k test-set patients at their own hospital only.

2. The expected use of thrombolysis in the 10k test-set for each hospital. The 10k patient data set is passed through all hospital models (by changing the one-hot hospital encoding).

In [ ]:
# Set up figure
fig = plt.figure(figsize=(6,6))

# Plot 1: actual vs subnet
ax1 = fig.add_subplot(111)
cmap = plt.cm.viridis
min_val = predicted_thrombolysis['10k_thrombolysis'].min()
max_val = predicted_thrombolysis['10k_thrombolysis'].max()
im = ax1.scatter(hospital_encoding['hosp_encode_x'],
            hospital_encoding['hosp_encode_y'],
            c=predicted_thrombolysis['10k_thrombolysis'],
            vmin=min_val, vmax=max_val, s=35, cmap=cmap)
cbar = plt.colorbar(im, ax=ax1)
cbar.set_label('Hospital thrombolysis use', rotation=90, labelpad=10)
ax1.set_xlabel('Hospital subnet output 1')
ax1.set_ylabel('Hospital subnet output 2')
plt.savefig('./output/hospital_subnet_scatter.jpg', dpi=300)
plt.show()

In this output, all hospitals appear to sit on $x=0$ or $y=1$, with use of thrombolysis in the hospital increasing along the $y$ axis and then the $x$ axis.

## Comparing clinical subnet activation with the number of hospitals that are predicted to give thrombolysis to each patient

In [ ]:
# Set up figure
fig = plt.figure(figsize=(6,6))

# Plot 1: actual vs subnet
cmap = plt.cm.viridis
ax1 = fig.add_subplot(111)
im = ax1.scatter(all_test['patient_encode_x'],
            all_test['patient_encode_y'],
            c=all_test['num_hosp_thrombolysing'],
            vmin=0, vmax=132, s=25, cmap=cmap, alpha=0.5)
cbar = plt.colorbar(im, ax=ax1)
cbar.set_label('Number of hospitals predicted to give thrombolysis', rotation=90, labelpad=10)
ax1.set_xlabel('Clinical subnet output 1')
ax1.set_ylabel('Clinical subnet output 2')
plt.savefig('./output/clinical_subnet_scatter.jpg', dpi=300)
plt.show()

The patients most likey to be given thrombolysis are those in the top left of the scatter plot ($x$ close to 0, and $y$ close to 1. Predicted use of therombolysis falls as a function of distance from that corner. Use of thrombolysis will also be effected by pathway data, but this scatter plot shows a dominant effect of clincal subnet output.

Patients appear to cluster mostly towards the edges of the plot.

In the following analysis, we will look at the characteristics of patients in different parts of the scatter plot.

### Analysis by stroke type

Plot the location of stroke type by haemorrhage (red) or infarction (blue).

In [ ]:
bleed = all_test['S2StrokeType_Primary Intracerebral Haemorrhage'] 

# Set up figure
cmap = plt.cm.bwr
fig = plt.figure(figsize=(6,6))

# Plot 1: actual vs subnet
ax1 = fig.add_subplot(111)
im = ax1.scatter(all_test['patient_encode_x'],
           all_test['patient_encode_y'],
           c=bleed,
        vmin=0, vmax=1, s=25, cmap=cmap, alpha=0.5)
#plt.colorbar(im, ax=ax1)
ax1.set_xlabel('Clinical subnet output 1')
ax1.set_ylabel('Clinical subnet output 2')

custom_lines = [Line2D([0], [0], color='r', marker='o', lw=0, markerfacecolor='r', markersize=8),
                Line2D([0], [0], color='b', marker='o', lw=0, markerfacecolor='b', markersize=8)]

plt.legend(custom_lines, ['Haemorrhagic stroke', 'Non-haemorrhagic stroke'],
          loc='upper center', framealpha=1)


plt.savefig('./output/clinical_subnet_scatter_stroke_type.jpg', dpi=300)
plt.show()

We observe that haemorrhagic stroke patients are clustered in one corner of the chart, suggesting that clinical subnet output may be used to compare similirity between patients (driven by reasons to give or not give thrombolysis).

## Summarise patients in corners of chart

Summarise key characteristics of patients in the four corners of the chart.

In [ ]:
paient_encode_regions = pd.DataFrame()

# Low x and low y
mask = (all_test['patient_encode_x'] < 0.2) & (all_test['patient_encode_y'] < 0.2)
paient_encode_regions['low_x_low_y'] = all_test[mask].mean()

# Low x and high y
mask = (all_test['patient_encode_x'] < 0.2) & (all_test['patient_encode_y'] > 0.8)
paient_encode_regions['low_x_high_y'] = all_test[mask].mean()

# High x and low y
mask = (all_test['patient_encode_x'] > 0.8) & (all_test['patient_encode_y'] <0.2)
paient_encode_regions['high_x_low_y'] = all_test[mask].mean()

# High x and high y
mask = (all_test['patient_encode_x'] > 0.8) & (all_test['patient_encode_y'] > 0.8)
paient_encode_regions['high_x_high_y'] = all_test[mask].mean()

In [ ]:
paient_encode_regions

In [ ]:
paient_encode_regions.to_csv('./output/paient_encode_regions.csv')

Show specific rows

In [ ]:
rows_to_show = [
    'S2RankinBeforeStroke',
    'S2NihssArrival',
    'S2StrokeType_Primary Intracerebral Haemorrhage',
    'num_hosp_thrombolysing']

paient_encode_regions.loc[rows_to_show].round(2)

We see different types of patients in different quadrants of the clinical subnet output scatter chart:

* *Low x and low y*: Largely non-thrombolysed patients with infarction stroke, and severe stroke and/or higher disability before stroke.

* *Low x and high y*: Largely thrombolysed patients.

* *High x and low y*: Largely non-thrombolysed patients with haemorrhagic stroke.

* *High x and high y*: Largely non-thrombolysed patients with infarction stroke and very mild stroke.

## Observations

2D clinical subnet analysis allows leads to potentially useful embedding/encoding of patients whereby similar patients are clustered together. 